#### Groud Truth

In [ ]:
import sys
import os

# Add the parent directory of `viewer` to the system path
sys.path.append(os.path.abspath(os.path.join("../..")))

from viewer.pdb_view import show_pdb_from_file

# Display the Trp-cage mini-protein (1L2Y)
show_pdb_from_file("1L2Y.pdb")

#### Unfolded Version

In [ ]:
show_pdb_from_file("1L2Y_unfold.pdb")

#### Simulate

In [ ]:
# Check if OpenMM is supported with GPU
from openmm import Platform
print([Platform.getPlatform(i).getName() for i in range(Platform.getNumPlatforms())])

In [ ]:
from openmm.app import PDBFile, ForceField, Simulation, Modeller, StateDataReporter, DCDReporter, HBonds, PME
from openmm import LangevinIntegrator, Platform
from openmm.unit import kelvin, picoseconds, nanometers, picosecond
import sys


def run_simulation(pdb_file):
    # Load the PDB file
    pdb = PDBFile(pdb_file)

    # Use the AMBER force field
    forcefield = ForceField('amber14-all.xml', 'amber14/tip3p.xml')
    
    # Create a system with explicit solvent (water)
    modeller = Modeller(pdb.topology, pdb.positions)
    modeller.addSolvent(forcefield, model='tip3p', padding=1.0*nanometers)

    # Create the system
    system = forcefield.createSystem(
        modeller.topology, 
        nonbondedMethod=PME, 
        nonbondedCutoff=1.0*nanometers, 
        constraints=HBonds
    )

    # Integrator setup (Langevin dynamics)
    integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)

    # Platform selection (CUDA for GPU)
    platform = Platform.getPlatformByName('CPU')

    # Simulation setup
    simulation = Simulation(modeller.topology, system, integrator, platform)
    simulation.context.setPositions(modeller.positions)

    # Save the topology to a PDB file
    with open('topology.pdb', 'w') as f:
        PDBFile.writeFile(modeller.topology, modeller.positions, f)

    # Minimize energy
    print('Minimizing...')
    simulation.minimizeEnergy()

    # Set up reporters to record simulation data
    simulation.reporters.append(StateDataReporter(sys.stdout, 1000, step=True, potentialEnergy=True, temperature=True))
    simulation.reporters.append(DCDReporter('trajectory.dcd', 1000))

    # Run the simulation
    print('Running Simulation...')
    simulation.step(20000)  # Number of steps to run, adjust as needed

    print('Simulation Complete!')
    
run_simulation("1L2Y_unfold.pdb")

In [ ]:
import os
import mdtraj as md


def visualize_md():
    os.makedirs("md_results", exist_ok=True)
    
    # Load the trajectory
    traj = md.load('trajectory.dcd', top="topology.pdb")

    for i in range(traj.n_frames):
        # Save each frame to a PDB file
        frame = traj[i]
    
        # Slice the frame to include only non-water atoms
        non_water_ion_atoms = frame.topology.select('not water and not (name Na or name Cl or resname NA or resname CL)')
        frame_removed = frame.atom_slice(non_water_ion_atoms)
        
        # Save the frame to a PDB file
        frame_removed.save_pdb(f"md_results/{i}.pdb")

visualize_md()